In [3]:
# import api keys
from api_keys import *
import pandas as pd

Using the Pandas Library, to read a Html and obtain the IATA and ICOA codes for the european airports

In [2]:
airports_codes = pd.read_html("https://airmundo.com/en/blog/airport-codes-european-airports/")

In [3]:
airports_codes_df = airports_codes[0]
airports_codes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Airport            264 non-null    object
 1   Country            264 non-null    object
 2   IATA airport code  264 non-null    object
 3   ICAO airport code  264 non-null    object
dtypes: object(4)
memory usage: 8.4+ KB


In [4]:
airports_codes_df.head()

,Airport,Country,IATA airport code,ICAO airport code
0,Tirana Airport,Albania,TIA,LATI
1,Yerevan Zvartnots Airport,Armenia,EVN,UDYZ
2,Graz Airport,Austria,GRZ,LOWG
3,Innsbruck Airport,Austria,INN,LOWI
4,Klagenfurt Airport,Austria,KLU,LOWK


Selecting the airports from wich we want to obtain data from. 

In [5]:
airport_DE = airports_codes_df.loc[airports_codes_df['Country'] == "United Kingdom"]
airport_DE

,Airport,Country,IATA airport code,ICAO airport code
239,Aberdeen Airport,United Kingdom,ABZ,EGPD
240,Belfast City Airport,United Kingdom,BHD,EGAC
241,Belfast International Airport,United Kingdom,BFS,EGAA
242,Birmingham Airport,United Kingdom,BHX,EGBB
243,Bristol Airport,United Kingdom,BRS,EGGD
244,Cardiff Airport,United Kingdom,CWL,EGFF
245,Doncaster Sheffield Airport,United Kingdom,DSA,EGCN
246,East Midlands Airport,United Kingdom,EMA,EGNX
247,Edinburgh Airport,United Kingdom,EDI,EGPH
248,Exeter Airport,United Kingdom,EXT,EGTE


Using the data from the airports searchs, we creates a csv file with data used as a in put for the APIs

In [4]:
api_input = pd.read_csv("https://raw.githubusercontent.com/icaromisquita/archives/main/Api_input.csv")
api_input

,Name,WikiData_code,ISO_3166_code,airport_icao
0,Berlin,Q64,DE,EDDB
1,London,Q84,GB,EGLC
2,Madrid,Q2807,ES,LEMD
3,Paris,Q90,FR,LFPG


The Icao codes needed in this project are

In [5]:
icao = api_input["airport_icao"].tolist()
icao

['EDDB', 'EGLC', 'LEMD', 'LFPG']

## Flights API

In [8]:
from datetime import datetime, timedelta

berlin_icoa = "EDDB"
to_local_time = datetime.now().strftime('%Y-%m-%dT%H:00')
from_local_time = (datetime.now() + timedelta(hours=11)).strftime('%Y-%m-%dT%H:00')
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{berlin_icoa}/{to_local_time}/{from_local_time}"

import requests

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flight_api_key
    }

berlin_flights= requests.request("GET", url, headers=headers, params=querystring)

from IPython.display import JSON
berlin_flights.json()

{'departures': [{'departure': {'scheduledTimeLocal': '2022-08-10 19:10+02:00',
    'scheduledTimeUtc': '2022-08-10 17:10Z',
    'terminal': '1',
    'quality': ['Basic']},
   'arrival': {'airport': {'name': 'Samsun'}, 'quality': []},
   'number': 'TK 1578',
   'status': 'Unknown',
   'codeshareStatus': 'Unknown',
   'isCargo': False,
   'aircraft': {'model': 'Airbus A320'},
   'airline': {'name': 'Turkish'}},
  {'departure': {'scheduledTimeLocal': '2022-08-10 18:45+02:00',
    'scheduledTimeUtc': '2022-08-10 16:45Z',
    'terminal': '1',
    'quality': ['Basic']},
   'arrival': {'airport': {'icao': 'EBBR', 'iata': 'BRU', 'name': 'Brussels'},
    'scheduledTimeLocal': '2022-08-10 20:10+02:00',
    'actualTimeLocal': '2022-08-10 20:10+02:00',
    'scheduledTimeUtc': '2022-08-10 18:10Z',
    'actualTimeUtc': '2022-08-10 18:10Z',
    'quality': ['Basic', 'Live']},
   'number': 'SN 2588',
   'status': 'Unknown',
   'codeshareStatus': 'Unknown',
   'isCargo': False,
   'aircraft': {'model': 

In [9]:
arrivals_berlin = berlin_flights.json()['arrivals']
berlin_icoa = "EDDB"

def get_flight_info(flight_json):
    # terminal
    try: terminal = flight_json['arrival']['terminal']
    except: terminal = None
    # aircraft
    try: aircraft = flight_json['aircraft']['model']
    except: aircraft = None

    return {
        'dep_airport':flight_json['departure']['airport']['name'],
        'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
        'terminal':terminal,
        'status':flight_json['status'],
        'aircraft':aircraft,
        'icao_code': berlin_icoa 
    }


# [get_flight_info(flight) for flight in arrivals_berlin]
arrivals_berlin = pd.DataFrame([get_flight_info(flight) for flight in arrivals_berlin])
arrivals_berlin

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Duesseldorf,2022-08-08 16:00+02:00,1,Unknown,Airbus A320,EDDB
1,Munich,2022-08-08 16:05+02:00,1,Unknown,Airbus A320,EDDB
2,Adana,2022-08-08 16:10+02:00,1,Unknown,Boeing 737,EDDB
3,Vienna,2022-08-08 16:15+02:00,1,Unknown,Airbus A320,EDDB
4,Nice,2022-08-08 16:15+02:00,1,Unknown,Airbus A320,EDDB
...,...,...,...,...,...,...
100,Paris,2022-08-08 22:40+02:00,1,Unknown,Airbus A220-300,EDDB
101,Madrid,2022-08-08 22:40+02:00,1,Unknown,Airbus A320,EDDB
102,Geneva,2022-08-08 22:40+02:00,1,Unknown,Airbus A320,EDDB
103,London,2022-08-08 22:50+02:00,1,Unknown,Airbus A320,EDDB


Making the calls for the others cities.

LONDON

In [10]:
london_icoa = icao[1]
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{london_icoa}/{to_local_time}/{from_local_time}"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

london_flights = requests.request("GET", url, headers=headers, params=querystring)

In [11]:
icao


['EDDB', 'EGLC', 'LEMD', 'LFPG ']

In [12]:
#LONDON
arrivals_london = london_flights.json()['arrivals']
london_icoa = icao[1]

def get_flight_info(flight_json):
    # terminal
    try: terminal = flight_json['arrival']['terminal']
    except: terminal = None
    # aircraft
    try: aircraft = flight_json['aircraft']['model']
    except: aircraft = None

    return {
        'dep_airport':flight_json['departure']['airport']['name'],
        'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
        'terminal':terminal,
        'status':flight_json['status'],
        'aircraft':aircraft,
        #'icao_code': london_icoa 
    }


# [get_flight_info(flight) for flight in arrivals_berlin]
arrivals_london = pd.DataFrame([get_flight_info(flight) for flight in arrivals_london])
arrivals_london

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Glasgow,2022-08-08 16:10+01:00,None,CanceledUncertain,Embraer 190,EGLC
1,Dublin,2022-08-08 16:30+01:00,None,Expected,Embraer 190,EGLC
2,Ibiza Town,2022-08-08 16:35+01:00,None,Expected,Embraer 190,EGLC
3,Berlin,2022-08-08 16:40+01:00,None,Expected,Embraer 190,EGLC
4,Menorca Island,2022-08-08 17:15+01:00,None,Expected,Embraer 190,EGLC
5,Amsterdam,2022-08-08 17:20+01:00,None,Expected,Embraer 190,EGLC
6,Castletown,2022-08-08 17:00+01:00,None,Expected,ATR 72,EGLC
7,Hondarribia,2022-08-08 17:20+01:00,None,Expected,Embraer 190,EGLC
8,Vilnius,2022-08-08 17:10+01:00,None,Expected,Embraer 190,EGLC
9,Luxembourg,2022-08-08 17:10+01:00,None,Expected,Bombardier Dash 8 Q400 / DHC-8-400,EGLC


MADRID

In [13]:
madrid_icoa = icao[2]
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{madrid_icoa}/{to_local_time}/{from_local_time}"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

madrid_flights= requests.request("GET", url, headers=headers, params=querystring)

In [14]:
#MADRID
arrivals_madrid = madrid_flights.json()['arrivals']
madrid_icoa = icao[2]

def get_flight_info(flight_json):
    # terminal
    try: terminal = flight_json['arrival']['terminal']
    except: terminal = None
    # aircraft
    try: aircraft = flight_json['aircraft']['model']
    except: aircraft = None

    return {
        'dep_airport':flight_json['departure']['airport']['name'],
        'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
        'terminal':terminal,
        'status':flight_json['status'],
        'aircraft':aircraft,
        'icao_code': madrid_icoa 
    }


# [get_flight_info(flight) for flight in arrivals_berlin]
arrivals_madrid = pd.DataFrame([get_flight_info(flight) for flight in arrivals_madrid])
arrivals_madrid

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Bogotá,2022-08-08 16:25+02:00,4S,Expected,Boeing 787-8,LEMD
1,Frankfurt-am-Main,2022-08-08 15:55+02:00,2,Expected,Airbus A320 (sharklets),LEMD
2,Frankfurt-am-Main,2022-08-08 15:55+02:00,2,Expected,Airbus A320 (sharklets),LEMD
3,Palma De Mallorca,2022-08-08 15:45+02:00,1,Expected,Boeing 737-800 (winglets),LEMD
4,Quito,2022-08-08 16:00+02:00,4S,Expected,Airbus A350-900,LEMD
...,...,...,...,...,...,...
609,Lanzarote Island,2022-08-09 00:25+02:00,4,Expected,Airbus A320,LEMD
610,Lisbon,2022-08-09 00:20+02:00,2,Expected,Embraer 190,LEMD
611,Lisbon,2022-08-09 00:20+02:00,2,Expected,Embraer 190,LEMD
612,Lanzarote Island,2022-08-09 00:25+02:00,4,Expected,Airbus A320,LEMD


PARIS

In [26]:
paris_icoa = icao[3]
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{paris_icoa}/{to_local_time}/{from_local_time}"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

paris_flights= requests.request("GET", url, headers=headers, params=querystring)

In [27]:
#PARIS
arrivals_paris = paris_flights.json()['arrivals']

def get_flight_info(flight_json):
    # terminal
    try: terminal = flight_json['arrival']['terminal']
    except: terminal = None
    # aircraft
    try: aircraft = flight_json['aircraft']['model']
    except: aircraft = None

    return {
        'dep_airport':flight_json['departure']['airport']['name'],
        'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
        'terminal':terminal,
        'status':flight_json['status'],
        'aircraft':aircraft,
        'icao_code': paris_icoa 
    }


# [get_flight_info(flight) for flight in arrivals_berlin]
arrivals_paris = pd.DataFrame([get_flight_info(flight) for flight in arrivals_paris])
arrivals_paris

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Toulouse-Blagnac,2022-08-08 16:05+02:00,2F,Unknown,Airbus A321,LFPG
1,Milan,2022-08-08 16:05+02:00,2D,Unknown,Bombardier CRJ1000,LFPG
2,Lyon,2022-08-08 16:10+02:00,2F,Unknown,Airbus A321,LFPG
3,Mykonos Island,2022-08-08 16:15+02:00,2F,Unknown,Airbus A319,LFPG
4,Algiers,2022-08-08 16:15+02:00,2E,Unknown,Airbus A321,LFPG
...,...,...,...,...,...,...
219,Olbia,2022-08-08 23:15+02:00,2B,Unknown,Airbus A320,LFPG
220,Tel Aviv Yafo,2022-08-08 23:20+02:00,2B,Unknown,Airbus A321,LFPG
221,Munich,2022-08-08 23:25+02:00,2B,Unknown,Airbus A320,LFPG
222,Split,2022-08-08 23:25+02:00,2B,Unknown,Airbus A321,LFPG


## Wheather API

### Using `pyowm` library

In [28]:
api_input

,Name,WikiData_code,ISO_3166_code,airport_icao
0,Berlin,Q64,DE,EDDB
1,London,Q84,GB,EGLC
2,Madrid,Q2807,ES,LEMD
3,Paris,Q90,FR,LFPG


In [32]:
city_name = api_input["Name"].tolist()
city_name

['Berlin', 'London', 'Madrid', 'Paris']

In [33]:
country_name = api_input["ISO_3166_code"].tolist()
country_name

['DE', 'GB', 'ES', 'FR']

### Connecting to OWM API

### For Berlin

In [34]:
city_name = ['Berlin', 'London', 'Madrid', 'Paris']
country_name = ['DE', 'GB', 'ES', 'FR']

response_berlin = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city_name[0]},{country_name[0]}&appid={OWM_key}&units=metric&lang=en')

from IPython.display import JSON
response_berlin.json()

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1659970800,
   'main': {'temp': 27.69,
    'feels_like': 26.88,
    'temp_min': 27.69,
    'temp_max': 27.84,
    'pressure': 1013,
    'sea_level': 1013,
    'grnd_level': 1018,
    'humidity': 30,
    'temp_kf': -0.15},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 1.35, 'deg': 319, 'gust': 1.82},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2022-08-08 15:00:00'},
  {'dt': 1659981600,
   'main': {'temp': 26.85,
    'feels_like': 26.37,
    'temp_min': 25.16,
    'temp_max': 26.85,
    'pressure': 1016,
    'sea_level': 1016,
    'grnd_level': 1018,
    'humidity': 31,
    'temp_kf': 1.69},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02d'}],
   'clouds': {'all': 14},
   'wind': {'speed': 3.52, 'deg': 2, 'gust': 4.69},
   'visibility': 10000,


In [35]:
#For Berlin
forecast_api = response_berlin.json()['list']
weather_info = []

# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city_name[0] + ',' + country_name[0] #Check it
    weather_info.append(weather_hour)    
    
weather_data_berlin = pd.DataFrame(weather_info)
weather_data_berlin.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-08-08 15:00:00,27.69,1.35,0.0,0,0,"Berlin,DE"
1,2022-08-08 18:00:00,26.85,3.52,0.0,0,0,"Berlin,DE"
2,2022-08-08 21:00:00,22.62,4.33,0.0,0,0,"Berlin,DE"
3,2022-08-09 00:00:00,17.89,3.18,0.0,0,0,"Berlin,DE"
4,2022-08-09 03:00:00,16.76,2.73,0.0,0,0,"Berlin,DE"


### For London


In [36]:
response_london = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city_name[1]},{country_name[1]}&appid={OWM_key}&units=metric&lang=en')

from IPython.display import JSON
response_london.json()

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1659970800,
   'main': {'temp': 27.91,
    'feels_like': 27.52,
    'temp_min': 27.91,
    'temp_max': 29.1,
    'pressure': 1026,
    'sea_level': 1026,
    'grnd_level': 1022,
    'humidity': 39,
    'temp_kf': -1.19},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02d'}],
   'clouds': {'all': 14},
   'wind': {'speed': 0.97, 'deg': 90, 'gust': 2.14},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2022-08-08 15:00:00'},
  {'dt': 1659981600,
   'main': {'temp': 26.74,
    'feels_like': 26.77,
    'temp_min': 24.4,
    'temp_max': 26.74,
    'pressure': 1026,
    'sea_level': 1026,
    'grnd_level': 1023,
    'humidity': 42,
    'temp_kf': 2.34},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02d'}],
   'clouds': {'all': 16},
   'wind': {'speed': 5.58, 'deg': 93, 'gust': 6.33},
   'visibility': 10000,

In [37]:
#For London
forecast_api = response_london.json()['list']
# look for the fields that could ve relevant: 
# better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day

weather_info = []

# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city_name[1] + ',' + country_name[1] 
    weather_info.append(weather_hour)    
    
weather_data_london = pd.DataFrame(weather_info)
weather_data_london.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-08-08 15:00:00,27.91,0.97,0.0,0,0,"London,GB"
1,2022-08-08 18:00:00,26.74,5.58,0.0,0,0,"London,GB"
2,2022-08-08 21:00:00,22.90,3.21,0.0,0,0,"London,GB"
3,2022-08-09 00:00:00,19.24,1.78,0.0,0,0,"London,GB"
4,2022-08-09 03:00:00,18.20,1.86,0.0,0,0,"London,GB"


### Madrid

In [38]:
response_madrid = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city_name[2]},{country_name[2]}&appid={OWM_key}&units=metric&lang=en')

from IPython.display import JSON
response_madrid.json()

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1659970800,
   'main': {'temp': 35.19,
    'feels_like': 33.03,
    'temp_min': 35.19,
    'temp_max': 35.4,
    'pressure': 1017,
    'sea_level': 1017,
    'grnd_level': 941,
    'humidity': 18,
    'temp_kf': -0.21},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 2.06, 'deg': 205, 'gust': 3.48},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2022-08-08 15:00:00'},
  {'dt': 1659981600,
   'main': {'temp': 35.15,
    'feels_like': 32.8,
    'temp_min': 35.08,
    'temp_max': 35.15,
    'pressure': 1015,
    'sea_level': 1015,
    'grnd_level': 940,
    'humidity': 16,
    'temp_kf': 0.07},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 2.02, 'deg': 194, 'gust': 3.59},
   'visibility': 10000,
   'p

In [39]:
#For Madrid
forecast_api = response_madrid.json()['list']
# look for the fields that could ve relevant: 
# better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day

weather_info = []

# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city_name[2] + ',' + country_name[2] 
    weather_info.append(weather_hour)    
    
weather_data_madrid = pd.DataFrame(weather_info)
weather_data_madrid.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-08-08 15:00:00,35.19,2.06,0.0,0.0,0,"Madrid,ES"
1,2022-08-08 18:00:00,35.15,2.02,0.0,0.0,0,"Madrid,ES"
2,2022-08-08 21:00:00,33.00,1.19,0.0,0.0,0,"Madrid,ES"
3,2022-08-09 00:00:00,28.09,4.50,0.0,0.0,0,"Madrid,ES"
4,2022-08-09 03:00:00,25.14,3.67,0.0,0.0,0,"Madrid,ES"


### Paris

In [40]:
response_paris = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city_name[3]},{country_name[3]}&appid={OWM_key}&units=metric&lang=en')

from IPython.display import JSON
response_paris.json()

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1659970800,
   'main': {'temp': 28.65,
    'feels_like': 27.59,
    'temp_min': 28.65,
    'temp_max': 30.51,
    'pressure': 1022,
    'sea_level': 1022,
    'grnd_level': 1017,
    'humidity': 30,
    'temp_kf': -1.86},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 0},
   'wind': {'speed': 4.75, 'deg': 28, 'gust': 5.86},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'd'},
   'dt_txt': '2022-08-08 15:00:00'},
  {'dt': 1659981600,
   'main': {'temp': 28.79,
    'feels_like': 27.55,
    'temp_min': 28.79,
    'temp_max': 29.06,
    'pressure': 1022,
    'sea_level': 1022,
    'grnd_level': 1017,
    'humidity': 27,
    'temp_kf': -0.27},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 4},
   'wind': {'speed': 4.73, 'deg': 22, 'gust': 6.38},
   'visibility': 10000,
  

In [41]:
#For Paris
forecast_api = response_paris.json()['list']
# look for the fields that could ve relevant: 
# better field descriptions https://www.weatherbit.io/api/weather-forecast-5-day

weather_info = []

# datetime, temperature, wind, prob_perc, rain_qty, snow = [], [], [], [], [], []
for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['datetime'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['prob_perc'] = float(forecast_3h['pop'])
    except: weather_hour['prob_perc'] = 0
    # rain
    try: weather_hour['rain_qty'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['rain_qty'] = 0
    # wind 
    try: weather_hour['snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['snow'] = 0
    weather_hour['municipality_iso_country'] = city_name[3] + ',' + country_name[3] #Check it
    weather_info.append(weather_hour)    
    
weather_data_paris = pd.DataFrame(weather_info)
weather_data_paris.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-08-08 15:00:00,28.65,4.75,0.0,0,0,"Paris,FR"
1,2022-08-08 18:00:00,28.79,4.73,0.0,0,0,"Paris,FR"
2,2022-08-08 21:00:00,25.93,5.00,0.0,0,0,"Paris,FR"
3,2022-08-09 00:00:00,20.65,4.53,0.0,0,0,"Paris,FR"
4,2022-08-09 03:00:00,18.33,3.65,0.0,0,0,"Paris,FR"


Joining the dataframes for the weather of the cities(Berlin, London, Madrid and Paris) into only one dataframe, named "weather_data" 

In [42]:
(
    weather_data = 
    pd.concat([weather_data_berlin, 
               weather_data_london, 
               weather_data_madrid, 
               weather_data_paris], 
              axis=0 ))

In [ ]:
weather_data.tail()

Dropping the old index and creating a new one

In [43]:
weather_data.reset_index(drop = True, inplace=True) 
weather_data

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-08-08 15:00:00,27.69,1.35,0.0,0.0,0,"Berlin,DE"
1,2022-08-08 18:00:00,26.85,3.52,0.0,0.0,0,"Berlin,DE"
2,2022-08-08 21:00:00,22.62,4.33,0.0,0.0,0,"Berlin,DE"
3,2022-08-09 00:00:00,17.89,3.18,0.0,0.0,0,"Berlin,DE"
4,2022-08-09 03:00:00,16.76,2.73,0.0,0.0,0,"Berlin,DE"
...,...,...,...,...,...,...,...
155,2022-08-13 00:00:00,27.03,2.79,0.0,0.0,0,"Paris,FR"
156,2022-08-13 03:00:00,24.31,1.90,0.0,0.0,0,"Paris,FR"
157,2022-08-13 06:00:00,24.12,2.36,0.0,0.0,0,"Paris,FR"
158,2022-08-13 09:00:00,31.24,3.57,0.0,0.0,0,"Paris,FR"


## Population data

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import unicodedata


cities = ['Berlin','London', 'Madrid','Paris']

def City_info(soup):
    
    ret_dict = {}
    ret_dict['city'] = soup.h1.get_text()
        
    if soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label') != None:
        i = soup.select_one('.mergedrow:-soup-contains("Mayor")>.infobox-label')
        mayor_name_html = i.find_next_sibling()
        mayor_name = unicodedata.normalize('NFKD',mayor_name_html.get_text())
        ret_dict['mayor']  = mayor_name
    
    if soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label') != None:
        j =  soup.select_one('.mergedrow:-soup-contains("City")>.infobox-label')
        area = j.find_next_sibling('td').get_text()
        ret_dict['city_size'] = unicodedata.normalize('NFKD',area)

    if soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data') != None:
        k = soup.select_one('.mergedtoprow:-soup-contains("Elevation")>.infobox-data')
        elevation_html = k.get_text()
        ret_dict['elevation'] = unicodedata.normalize('NFKD',elevation_html)
    
    if soup.select_one('.mergedtoprow:-soup-contains("Population")') != None:
        l = soup.select_one('.mergedtoprow:-soup-contains("Population")')
        c_pop = l.findNext('td').get_text()
        ret_dict['city_population'] = c_pop
    
    if soup.select_one('.infobox-label>[title^=Urban]') != None:
        m = soup.select_one('.infobox-label>[title^=Urban]')
        u_pop = m.findNext('td')
        ret_dict['urban_population'] = u_pop.get_text()

    if soup.select_one('.infobox-label>[title^=Metro]') != None:
        n = soup.select_one('.infobox-label>[title^=Metro]')
        m_pop = n.findNext('td')
        ret_dict['metro_population'] = m_pop.get_text()
    
    if soup.select_one('.latitude') != None:
        o = soup.select_one('.latitude')
        ret_dict['lat'] = o.get_text()

    if soup.select_one('.longitude') != None:    
        p = soup.select_one('.longitude')
        ret_dict['long'] = p.get_text()
    
    return ret_dict

list_of_city_info = []
for city in cities:
    url = 'https://en.wikipedia.org/wiki/{}'.format(city)
    web = requests.get(url,'html.parser')
    soup = bs(web.content)
    list_of_city_info.append(City_info(soup))
df_cities = pd.DataFrame(list_of_city_info)
# df_cities = df_cities.set_index('city')
df_cities

,city,mayor,city_size,elevation,city_population,urban_population,metro_population,lat,long
0,Berlin,Franziska Giffey (SPD),891.7 km2 (344.3 sq mi),34 m (112 ft),"3,769,495","4,473,101","6,144,600",52°31′12″N,13°24′18″E
1,London,Greater London Authority• Mayor Sadiq Khan (L)...,Greater LondonCity of London,36 ft (11 m),"9,002,488[1]","9,950,000","14,257,962[4] (1st)",51°30′26″N,0°7′39″W
2,Madrid,José Luis Martínez-Almeida (PP),NaN,"650 m (2,130 ft)","3,223,334","6,211,000[2]","6,791,667[1]",40°25′00″N,03°42′09″W
3,Paris,Anne Hidalgo (PS),NaN,28–131 m (92–430 ft) (avg. 78 m or 256 ft),"2,165,423","10,858,852","13,024,518",48°51′24″N,2°21′08″E


## Airports data

In [45]:
import pandas as pd

airports_cities = (
pd.read_csv('airports.csv')
    .query('type == "large_airport"')
    .filter(['name','latitude_deg','longitude_deg','iso_country','iso_region','municipality','gps_code','iata_code'])
    .rename(columns={'gps_code':'icao_code'})
    .assign(municipality_iso_country = lambda x: x['municipality'] + ',' + x['iso_country'])
)
airports_cities.head()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"


In [46]:
airports_cities.query('municipality == "Berlin"')

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
20244,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE"


## Check the tables

In [47]:
arrivals_berlin.head()

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Duesseldorf,2022-08-08 16:00+02:00,1,Unknown,Airbus A320,EDDB
1,Munich,2022-08-08 16:05+02:00,1,Unknown,Airbus A320,EDDB
2,Adana,2022-08-08 16:10+02:00,1,Unknown,Boeing 737,EDDB
3,Vienna,2022-08-08 16:15+02:00,1,Unknown,Airbus A320,EDDB
4,Nice,2022-08-08 16:15+02:00,1,Unknown,Airbus A320,EDDB


In [64]:
arrivals_london.head()

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Glasgow,2022-08-08 16:10+01:00,None,CanceledUncertain,Embraer 190,EGLC
1,Dublin,2022-08-08 16:30+01:00,None,Expected,Embraer 190,EGLC
2,Ibiza Town,2022-08-08 16:35+01:00,None,Expected,Embraer 190,EGLC
3,Berlin,2022-08-08 16:40+01:00,None,Expected,Embraer 190,EGLC
4,Menorca Island,2022-08-08 17:15+01:00,None,Expected,Embraer 190,EGLC


Concatenating arrivals data for all airports

In [65]:
(
 arrivals_data = 
 pd.concat([arrivals_berlin, 
            arrivals_london, 
            arrivals_madrid, 
            arrivals_paris], axis=0 )
)
arrivals_data.reset_index(drop = True, inplace=True) 
arrivals_data

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Duesseldorf,2022-08-08 16:00+02:00,1,Unknown,Airbus A320,EDDB
1,Munich,2022-08-08 16:05+02:00,1,Unknown,Airbus A320,EDDB
2,Adana,2022-08-08 16:10+02:00,1,Unknown,Boeing 737,EDDB
3,Vienna,2022-08-08 16:15+02:00,1,Unknown,Airbus A320,EDDB
4,Nice,2022-08-08 16:15+02:00,1,Unknown,Airbus A320,EDDB
...,...,...,...,...,...,...
972,Olbia,2022-08-08 23:15+02:00,2B,Unknown,Airbus A320,LFPG
973,Tel Aviv Yafo,2022-08-08 23:20+02:00,2B,Unknown,Airbus A321,LFPG
974,Munich,2022-08-08 23:25+02:00,2B,Unknown,Airbus A320,LFPG
975,Split,2022-08-08 23:25+02:00,2B,Unknown,Airbus A321,LFPG


In [48]:
cities = airports_cities.filter(['municipality','iso_country','municipality_iso_country']).drop_duplicates()
cities.head()

,municipality,iso_country,municipality_iso_country
10890,Honiara,SB,"Honiara,SB"
12461,Port Moresby,PG,"Port Moresby,PG"
12981,Reykjavík,IS,"Reykjavík,IS"
13028,Prishtina,XK,"Prishtina,XK"
17254,"Xintai, Tai'an",CN,"Xintai, Tai'an,CN"


In [49]:
arrivals_berlin.head()

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
0,Duesseldorf,2022-08-08 16:00+02:00,1,Unknown,Airbus A320,EDDB
1,Munich,2022-08-08 16:05+02:00,1,Unknown,Airbus A320,EDDB
2,Adana,2022-08-08 16:10+02:00,1,Unknown,Boeing 737,EDDB
3,Vienna,2022-08-08 16:15+02:00,1,Unknown,Airbus A320,EDDB
4,Nice,2022-08-08 16:15+02:00,1,Unknown,Airbus A320,EDDB


In [70]:
arrivals_data = airports_cities.merge(arrivals_data, on='icao_code', how='inner').merge(weather_data, on='municipality_iso_country', how='inner').head()

In [71]:
arrivals_data.tail()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country,dep_airport,sched_arr_loc_time,terminal,status,aircraft,datetime,temperature,wind,prob_perc,rain_qty,snow
0,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Duesseldorf,2022-08-08 16:00+02:00,1,Unknown,Airbus A320,2022-08-08 15:00:00,27.69,1.35,0.0,0.0,0
1,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Duesseldorf,2022-08-08 16:00+02:00,1,Unknown,Airbus A320,2022-08-08 18:00:00,26.85,3.52,0.0,0.0,0
2,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Duesseldorf,2022-08-08 16:00+02:00,1,Unknown,Airbus A320,2022-08-08 21:00:00,22.62,4.33,0.0,0.0,0
3,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Duesseldorf,2022-08-08 16:00+02:00,1,Unknown,Airbus A320,2022-08-09 00:00:00,17.89,3.18,0.0,0.0,0
4,Berlin Brandenburg Airport,52.351389,13.493889,DE,DE-BR,Berlin,EDDB,BER,"Berlin,DE",Duesseldorf,2022-08-08 16:00+02:00,1,Unknown,Airbus A320,2022-08-09 03:00:00,16.76,2.73,0.0,0.0,0


In [ ]:
df_cities

In [67]:
'''
df_cities['municipality_iso_country'] = [
    'Berlin,DE',
    'Paris,FR',
    'Amsterdam,NL',
    'Barcelona,ES',
    'Rome,IT',
    'Lisbon,PT',
    'Prague,CZE',
    'London,GB',
    'Madrid,ES'
]
'''

In [ ]:
df_cities['municipality_iso_country'] = [
    'Berlin,DE',
    'London,GB',
    'Madrid,ES',
    'Paris,FR'    
]

## Update data into database

In [54]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   datetime                  160 non-null    object 
 1   temperature               160 non-null    float64
 2   wind                      160 non-null    float64
 3   prob_perc                 160 non-null    float64
 4   rain_qty                  160 non-null    float64
 5   snow                      160 non-null    int64  
 6   municipality_iso_country  160 non-null    object 
dtypes: float64(4), int64(1), object(2)
memory usage: 8.9+ KB


### `sqlalchemy`

#### Establish the connection

In [81]:
import pandas as pd
# import sqlalchemy

In [56]:
schema="gans"
host="127.0.0.1"
user="root"
password="Ud73QkMaYB2MWbD"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

#### Update the tables

ARRIVALS

In [87]:
import numpy as np
(
arrivals_berlin
    .replace({np.nan},'unknown')
    .assign(sched_arr_loc_time = lambda x: pd.to_datetime(x['sched_arr_loc_time']))
    .to_sql('arrivals', if_exists='append', con=con, index=False))

In [61]:
arrivals_berlin.tail()

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,icao_code
100,Paris,2022-08-08 22:40+02:00,1,Unknown,Airbus A220-300,EDDB
101,Madrid,2022-08-08 22:40+02:00,1,Unknown,Airbus A320,EDDB
102,Geneva,2022-08-08 22:40+02:00,1,Unknown,Airbus A320,EDDB
103,London,2022-08-08 22:50+02:00,1,Unknown,Airbus A320,EDDB
104,Amsterdam,2022-08-08 22:55+02:00,1,Unknown,Airbus A319,EDDB


AIRPORTS

In [72]:
airports_cities.head()

,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,icao_code,iata_code,municipality_iso_country
10890,Honiara International Airport,-9.428000,160.054993,SB,SB-CT,Honiara,AGGH,HIR,"Honiara,SB"
12461,Port Moresby Jacksons International Airport,-9.443380,147.220001,PG,PG-NCD,Port Moresby,AYPY,POM,"Port Moresby,PG"
12981,Keflavik International Airport,63.985001,-22.605600,IS,IS-2,Reykjavík,BIKF,KEF,"Reykjavík,IS"
13028,Priština Adem Jashari International Airport,42.572800,21.035801,XK,XK-01,Prishtina,BKPR,PRN,"Prishtina,XK"
17254,Guodu Air Base,36.001741,117.632010,CN,CN-37,"Xintai, Tai'an",NaN,NaN,"Xintai, Tai'an,CN"


In [84]:
(
    airports_cities
        .dropna()
        .to_sql('airports', if_exists='append', con=con, index=False)
)

CITIES #change to my actual cities

In [73]:
df_cities.head()

,city,mayor,city_size,elevation,city_population,urban_population,metro_population,lat,long,municipality_iso_country
0,Berlin,Franziska Giffey (SPD),891.7 km2 (344.3 sq mi),34 m (112 ft),"3,769,495","4,473,101","6,144,600",52°31′12″N,13°24′18″E,"Berlin,DE"
1,Paris,Anne Hidalgo (PS),NaN,28–131 m (92–430 ft) (avg. 78 m or 256 ft),"2,165,423","10,858,852","13,024,518",48°51′24″N,2°21′08″E,"Paris,FR"
2,Amsterdam,Femke Halsema (GL),NaN,−2 m (−7 ft),"907,976","1,558,755",NaN,52°22′N,4°54′E,"Amsterdam,NL"
3,Barcelona,Ada Colau Ballano[1] (Barcelona en Comú),101.4 km2 (39.2 sq mi),12 m (39 ft),"1,620,343","4,840,000[3]","5,474,482[4]",41°23′N,2°11′E,"Barcelona,ES"
4,Rome,Strong Mayor–Council,"4,342,212[2]",21 m (69 ft),1st in Italy (3rd in the EU),NaN,Rome Capital,41°53′36″N,12°28′58″E,"Rome,IT"


In [85]:
(
df_cities
    .dropna()
    .rename(
        columns={
            'lat':'latitude',
            'long':'longitude'
            }
        )
    .to_sql('cities', con=con, if_exists='append', index=False)
    )

WEATHER

In [86]:
(
    weather_data
        .assign(datetime = lambda x: pd.to_datetime(x['datetime']))
        .to_sql('weather', if_exists='append', con=con, index=False)
)

In [75]:
weather_data.head()

,datetime,temperature,wind,prob_perc,rain_qty,snow,municipality_iso_country
0,2022-08-08 15:00:00,27.69,1.35,0.0,0.0,0,"Berlin,DE"
1,2022-08-08 18:00:00,26.85,3.52,0.0,0.0,0,"Berlin,DE"
2,2022-08-08 21:00:00,22.62,4.33,0.0,0.0,0,"Berlin,DE"
3,2022-08-09 00:00:00,17.89,3.18,0.0,0.0,0,"Berlin,DE"
4,2022-08-09 03:00:00,16.76,2.73,0.0,0.0,0,"Berlin,DE"
